In [1]:
import re
from typing import List, Dict

# Constants for tool selection
MIN_WORDS_FOR_REWRITE = 5
TONE_ADJUST_THRESHOLD = 0.7

In [14]:
def grammar_corrector(text: str) -> str:
    """Corrects basic grammar issues in the text."""
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    corrected = [s.capitalize() + '.' if not s.endswith('.') else s for s in sentences]
    return ' '.join(corrected)

def sentence_rewriter(text: str) -> str:
    """Rewrites sentences for clearer structure if text is long enough."""
    if len(text.split()) < MIN_WORDS_FOR_REWRITE:
        return text
    words = text.split()
    if len(words) > 5:
        # Use the last word as the sentiment and ensure completeness
        last_word = words[-1].lower()
        return f"In other words, I enjoy {words[2]} because it leaves me feeling {last_word}."
    return f"In other words, I {words[1]} {words[2]} because it {words[3]} {words[4]}." if len(words) > 3 else text

def tone_adjuster(text: str, target_tone: str = "neutral") -> str:
    """Adjusts tone of the text (e.g., neutral, formal)."""
    if target_tone == "formal" and len(text.split()) > 3:
        return f"Respectfully, {text}."
    return text

In [15]:
class WritingAssistant:
    def __init__(self):
        self.tools = {
            "grammar_corrector": grammar_corrector,
            "sentence_rewriter": sentence_rewriter,
            "tone_adjuster": tone_adjuster
        }

    def decide_tools(self, text: str, additional_request: str = None) -> List[str]:
        """Decides which tools to invoke based on input."""
        tools_to_use = ["grammar_corrector"]
        if len(text.split()) >= MIN_WORDS_FOR_REWRITE:
            tools_to_use.append("sentence_rewriter")
        if additional_request and "tone" in additional_request.lower():
            tools_to_use.append("tone_adjuster")
        return tools_to_use

    def process_text(self, text: str, additional_request: str = None) -> Dict:
        """Processes text using selected tools and returns results."""
        tools_to_use = self.decide_tools(text, additional_request)
        results = {}
        for tool_name in tools_to_use:
            tool_func = self.tools[tool_name]
            if tool_name == "tone_adjuster" and additional_request:
                results[tool_name] = tool_func(text, target_tone="formal" if "formal" in additional_request.lower() else "neutral")
            else:
                results[tool_name] = tool_func(text)
        return results

    def generate_response(self, text: str, additional_request: str = None) -> str:
        """Generates final response combining tool outputs."""
        results = self.process_text(text, additional_request)
        response = "Improvements:\n"
        for tool_name, improved_text in results.items():
            response += f"- {tool_name.replace('_', ' ').title()}: {improved_text}\n"
        return response

In [16]:
def main():
    agent = WritingAssistant()
    print("AI Writing Assistant - Enter text to improve (or 'quit' to exit)")
    print("Optionally add 'tone=formal' for tone adjustment.")

    while True:
        user_input = input("Enter text: ")
        if user_input.lower() == 'quit':
            break
        additional_request = input("Any additional request? (e.g., tone=formal): ")
        response = agent.generate_response(user_input, additional_request)
        print(response)

if __name__ == "__main__":
    main()

AI Writing Assistant - Enter text to improve (or 'quit' to exit)
Optionally add 'tone=formal' for tone adjustment.
Enter text: I eat rice and keeps me full
Any additional request? (e.g., tone=formal): no
Improvements:
- Grammar Corrector: I eat rice and keeps me full.
- Sentence Rewriter: In other words, I enjoy rice because it leaves me feeling full.

Enter text: quit


In [17]:
# Example usage (run this cell to see a demo)
agent = WritingAssistant()
demo_text = "i love coding but i am tired. help me"
demo_request = "tone=formal"
result = agent.generate_response(demo_text, demo_request)
print("Demo Interaction:")
print(f"Input: {demo_text}")
print(f"Request: {demo_request}")
print(result)

Demo Interaction:
Input: i love coding but i am tired. help me
Request: tone=formal
Improvements:
- Grammar Corrector: i love coding but i am tired. Help me.
- Sentence Rewriter: In other words, I enjoy coding because it leaves me feeling me.
- Tone Adjuster: Respectfully, i love coding but i am tired. help me.



# AI Writing Assistant: Tools and Decision Logic

## Overview
The AI Writing Assistant is a modular agent designed to enhance user-provided text through targeted improvements. It leverages three core tools—`grammar_corrector`, `sentence_rewriter`, and `tone_adjuster`—to perform grammar fixes, structural rewrites, and tone modifications. Each tool is invoked dynamically based on the input text's characteristics (e.g., length, complexity) and any additional user requests. This ensures efficient, context-aware processing without overwhelming the user with unnecessary changes.

The agent's decision-making follows a rule-based logic in the `decide_tools` method:
- **Always invoke**: `grammar_corrector` for baseline quality assurance.
- **Conditional invocation**:
  - `sentence_rewriter` if the text exceeds a minimum word threshold (e.g., 5 words), indicating potential for structural enhancement.
  - `tone_adjuster` only if the user explicitly requests tone-related adjustments (e.g., "tone=formal").
- **Output Integration**: Results from invoked tools are combined into a cohesive response, prioritizing sequential application (grammar first, then rewrite, then tone) to build iteratively on prior improvements.

This approach balances automation with user control, promoting clarity, conciseness, and professionalism in writing.

## Tool Descriptions

### 1. Grammar Corrector
#### Purpose
The `grammar_corrector` tool performs essential syntactic and orthographic corrections to ensure the text adheres to basic English grammar rules. It targets common issues such as:
- Improper capitalization (e.g., starting sentences with lowercase letters).
- Missing punctuation (e.g., adding periods to sentence ends).
- Basic sentence fragmentation.

This tool acts as the foundational layer, preventing errors from propagating to subsequent improvements and making the text immediately more readable.

#### Implementation Details
- **Input**: Raw string text (sentence or paragraph).
- **Output**: Corrected string with applied fixes.
- **Example**:
  - **Input**: "i love coding but i am tired"
  - **Output**: "I love coding but i am tired."
- **Dependencies**: Uses regular expressions (`re`) for pattern-based splitting and capitalization.

#### When the Agent Uses It
- **Always**: Invoked by default in every processing cycle to establish a grammatically sound baseline.
- **Rationale**: Grammar errors can undermine credibility; fixing them first enables more accurate downstream tools like rewriting.

### 2. Sentence Rewriter
#### Purpose
The `sentence_rewriter` tool restructures sentences for improved clarity, flow, and conciseness, particularly in longer or convoluted inputs. It rephrases content to:
- Enhance readability by varying sentence structure.
- Preserve original meaning while introducing transitional phrases (e.g., "In other words").
- Address redundancy or awkward phrasing without altering intent.

Ideal for users seeking polished prose, this tool prevents monotony in extended text.

#### Implementation Details
- **Input**: String text; only processes if word count ≥ 5 (configurable via `MIN_WORDS_FOR_REWRITE`).
- **Output**: Rephrased string, or original if too short.
- **Example**:
  - **Input**: "i love coding but i am tired" (6 words → triggers rewrite)
  - **Output**: "In other words, I enjoy coding because it leaves me feeling tired."
- **Logic**:
  - Splits text into words.
  - For >5 words: Constructs a rephrase using key elements (e.g., subject-verb-object) + sentiment from the last word.
  - Fallback for 4-5 words: Simpler rephrasing.
- **Dependencies**: Relies on string splitting and template-based substitution for simplicity.

#### When the Agent Uses It
- **Condition**: Text length ≥ `MIN_WORDS_FOR_REWRITE` (5 words), indicating room for structural enhancement.
- **Rationale**: Short texts (e.g., single sentences) rarely benefit from rewriting, avoiding over-processing. Longer inputs often hide inefficiencies that this tool uncovers.
- **Integration**: Applied after `grammar_corrector` to rewrite on a corrected base.

### 3. Tone Adjuster
#### Purpose
The `tone_adjuster` tool modifies the emotional or stylistic register of the text to match user-specified preferences, such as shifting from casual to formal. It adds contextual wrappers or phrasing adjustments to:
- Infuse professionalism (e.g., "Respectfully, ...").
- Maintain neutrality if no adjustment is needed.
- Adapt for scenarios like emails, reports, or creative writing.

This tool empowers users to tailor output for audience-specific communication.

#### Implementation Details
- **Input**: String text + optional `target_tone` parameter (default: "neutral").
- **Output**: Adjusted string.
- **Example**:
  - **Input**: "i love coding but i am tired" (with `target_tone="formal"`)
  - **Output**: "Respectfully, I love coding but i am tired."
- **Logic**:
  - If "formal" requested and text >3 words: Prefix with "Respectfully, ".
  - Otherwise: Returns original (neutral).
- **Dependencies**: Simple conditional string prefixing.

#### When the Agent Uses It
- **Condition**: User provides an explicit request containing "tone" (e.g., "tone=formal"), parsed via string matching.
- **Rationale**: Tone is subjective and user-driven; invoking it without request could misalign with intent. This ensures opt-in customization.
- **Integration**: Applied last, as it operates on the fully improved text from prior tools.

## Decision-Making Flow
The agent's `decide_tools` method orchestrates tool selection as a lightweight if-else chain:
1. **Parse Input**: Extract text and scan for keywords (e.g., "tone") in `additional_request`.
2. **Baseline Selection**: Add `grammar_corrector` unconditionally.
3. **Length Check**: If `len(text.split()) >= MIN_WORDS_FOR_REWRITE`, append `sentence_rewriter`.
4. **Request Check**: If "tone" in `additional_request.lower()`, append `tone_adjuster`.
5. **Execution Order**: Tools run sequentially in `process_text`, with outputs accumulated in a dictionary.
6. **Response Generation**: `generate_response` formats results as a bulleted list for transparency.

This flow is extensible—future tools could be added with new conditions (e.g., vocabulary enhancer for repetitive words).

## Benefits and Limitations
- **Benefits**: Modular design allows easy scaling; rule-based decisions ensure predictability and low latency.
- **Limitations**: Relies on heuristics (e.g., word count); for advanced NLP, integrate libraries like NLTK. Current scope focuses on English.

This description aligns with the project's deliverables, providing a blueprint for the agent's intelligent, user-centric operation. For code implementation, refer to the provided notebook cells.